In [1]:
# 步骤如下：
# a) 分词
# b) 词语-> id
# matrix -> [ v, embed_size ]
# 词语A -> id(5)
# 词表
# c) label -> id

import sys
import os
import jieba  # pip install jieba

# input file
train_file = './cnews_data/cnews.train.txt'
val_file = './cnews_data/cnews.val.txt'
test_file = './cnews_data/cnews.test.txt'

# output file
seg_train_file = './cnews_data/cnews.train.seg.txt'
seg_val_file = './cnews_data/cnews.val.seg.txt'
seg_test_file = './cnews_data/cnews.test.seg.txt'

vocab_file = './cnews_data/cnews.vocab.txt'
category_file = './cnews_data/cnews.category.txt'




In [6]:
with open(val_file, 'r') as f:
    lines = f.readlines()
    
# python2需要先将内容进行转码，从utf8转到unicode
label, content = lines[0].strip('\r\n').split('\t') # strip()是去掉结尾开头的相应的字符

word_iter = jieba.cut(content)

print(content)
print('/ '.join(word_iter))


Building prefix dict from the default dictionary ...


黄蜂vs湖人首发：科比带伤战保罗 加索尔救赎之战 新浪体育讯北京时间4月27日，NBA季后赛首轮洛杉矶湖人主场迎战新奥尔良黄蜂，此前的比赛中，双方战成2-2平，因此本场比赛对于两支球队来说都非常重要，赛前双方也公布了首发阵容：湖人队：费舍尔、科比、阿泰斯特、加索尔、拜纳姆黄蜂队：保罗、贝里内利、阿里扎、兰德里、奥卡福[新浪NBA官方微博][新浪NBA湖人新闻动态微博][新浪NBA专题][黄蜂vs湖人图文直播室](新浪体育)


Dumping model to file cache C:\Users\i076453\AppData\Local\Temp\jieba.cache
Loading model cost 0.900 seconds.
Prefix dict has been built succesfully.


黄蜂/ vs/ 湖人/ 首发/ ：/ 科比/ 带伤/ 战/ 保罗/  / 加索尔/ 救赎/ 之战/  / 新浪/ 体育讯/ 北京/ 时间/ 4/ 月/ 27/ 日/ ，/ NBA/ 季后赛/ 首轮/ 洛杉矶/ 湖人/ 主场/ 迎战/ 新奥尔良/ 黄蜂/ ，/ 此前/ 的/ 比赛/ 中/ ，/ 双方/ 战成/ 2/ -/ 2/ 平/ ，/ 因此/ 本场/ 比赛/ 对于/ 两支/ 球队/ 来说/ 都/ 非常/ 重要/ ，/ 赛前/ 双方/ 也/ 公布/ 了/ 首发/ 阵容/ ：/ 湖人队/ ：/ 费舍尔/ 、/ 科比/ 、/ 阿泰斯特/ 、/ 加索尔/ 、/ 拜纳姆/ 黄蜂队/ ：/ 保罗/ 、/ 贝里/ 内利/ 、/ 阿里/ 扎/ 、/ 兰德/ 里/ 、/ 奥卡福/ [/ 新浪/ NBA/ 官方/ 微博/ ]/ [/ 新浪/ NBA/ 湖人/ 新闻动态/ 微博/ ]/ [/ 新浪/ NBA/ 专题/ ]/ [/ 黄蜂/ vs/ 湖人/ 图文/ 直播室/ ]/ (/ 新浪/ 体育/ )


In [7]:
# 实际处理不能将所有的内容全部读入内存，要分批读入

def generate_seg_file(input_file, output_seg_file):
    """segement sentences in each line in input file"""
    with open(input_file, 'r') as f:
        lines = f.readlines() # 一次性读取所有数据到内存
    with open(output_seg_file, 'w') as f:
        for line in lines:
            label, content = line.strip('\r\n').split('\t')
            word_iter = jieba.cut(content)
            word_content = ''
            for word in word_iter:
                word = word.strip()
                if word != '':
                    word_content += word + ' '
            out_line = '%s\t%s\n' % (label, word_content.strip(' ')) # strip(' ') 去掉最后的空格
            f.write(out_line)
    
generate_seg_file(train_file, seg_train_file)
generate_seg_file(val_file, seg_val_file)
generate_seg_file(test_file, seg_test_file)

In [9]:
# 生成词表
def generate_vocab(input_seg_file, output_vocab_file):
    """generate vocab file"""
    with open(input_seg_file, 'r') as f:
        lines = f.readlines()
    word_dict = {}
    for line in lines:
        label, content = line.strip('\r\n').split('\t')
        # 统计频次
        for word in content.split(): # split()默认用空格分割
            word_dict.setdefault(word, 0)
            word_dict[word] += 1
    # [(word, frequency),....,()]
    sorted_word_dict = sorted(word_dict.items(), key = lambda d:d[1], reverse=True)
    
    with open(output_vocab_file, 'w') as f:
        f.write('<UNK>\t10000000\n')
        for item in sorted_word_dict:
            f.write('%s\t%d\n' % (item[0], item[1]))
            
generate_vocab(seg_train_file, vocab_file)
    

In [10]:
# 生成label文件
def generate_category_file(input_file, category_file):
    """generate category file"""
    with open(input_file, 'r') as f:
        lines = f.readlines()
    category_dict = {}
    for line in lines:
        label, content = line.strip('\r\n').split('\t')
        category_dict.setdefault(label, 0)
        category_dict[label] += 1
    category_number = len(category_dict)
    with open(category_file, 'w') as f:
        for category in category_dict:
            line = '%s\n' % category
            print('%s\t%d' % (category, category_dict[category]))
            f.write(line)
            
generate_category_file(train_file, category_file)

体育	5000
娱乐	5000
家居	5000
房产	5000
教育	5000
时尚	5000
时政	5000
游戏	5000
科技	5000
财经	5000
